In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("Order2.csv")

In [8]:
df.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Transaction List
transactions = []
for i in range(len(df)):
    transaction = []

    for item in df.values[i]:
        if not pd.isna(item):
            transaction.append(str(item).strip())

    transactions.append(transaction)

print(transactions[0:5])
print(len(transactions))

[['burgers', 'meatballs', 'eggs'], ['chutney'], ['turkey', 'avocado'], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea'], ['low fat yogurt']]
7500


In [10]:
#As Apriori algorithm needs encoding so we will apply OHE
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_onehot = pd.DataFrame(te_ary,columns=te.columns_)

df_onehot.head()

,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
from mlxtend.frequent_patterns import apriori, association_rules

In [12]:
#First, calculate minimum support using the formula....
min_support = 50 / len(transactions)

#Second, train Apriori
frequent_itemsets = apriori(df_onehot,min_support=min_support,use_colnames=True)

print(frequent_itemsets.sort_values(by='support',ascending=False).head(10))

     support             itemsets
56  0.238267      (mineral water)
25  0.179733               (eggs)
75  0.174133          (spaghetti)
31  0.170933       (french fries)
18  0.163867          (chocolate)
42  0.132000          (green tea)
55  0.129600               (milk)
43  0.098267        (ground beef)
37  0.095333  (frozen vegetables)
63  0.095067           (pancakes)


In [13]:
#Generate rules (considering lift here for positive relation)
rules = association_rules(frequent_itemsets,metric="lift",min_threshold=1.0)

#Sorting rules by confidence and then by lift for strong and meaningful rules only
rules_sorted = rules.sort_values(by=['confidence','lift'],ascending=False)

print(rules_sorted.head(10))

                          antecedents      consequents  antecedent support  \
939                      (milk, soup)  (mineral water)            0.015200   
837  (ground beef, frozen vegetables)  (mineral water)            0.016933   
982                 (spaghetti, soup)  (mineral water)            0.014267   
589              (chocolate, chicken)  (mineral water)            0.014667   
908           (ground beef, pancakes)  (mineral water)            0.014533   
843  (ground beef, frozen vegetables)      (spaghetti)            0.016933   
779               (ground beef, eggs)  (mineral water)            0.020000   
970               (spaghetti, salmon)  (mineral water)            0.013467   
699            (chocolate, olive oil)  (mineral water)            0.016400   
890               (ground beef, milk)  (mineral water)            0.022000   

     consequent support   support  confidence      lift  representativity  \
939            0.238267  0.008533    0.561404  2.356198         

In [14]:
item_in = input("Enter an item to find rules for (e.g., milk): ").strip()

item_set = frozenset({item_in})

matching_rules = rules_sorted[rules_sorted['antecedents'] == item_set]

if matching_rules.empty:
    print(f"\nNo rules found where '{item_in}' is the only item.")
else:
    print(f"\n--- Rules for 'IF customer buys {item_in}' ---")
    print(matching_rules)

Enter an item to find rules for (e.g., milk):  milk



--- Rules for 'IF customer buys milk' ---
    antecedents                         consequents  antecedent support  \
444      (milk)                     (mineral water)              0.1296   
456      (milk)                         (spaghetti)              0.1296   
156      (milk)                         (chocolate)              0.1296   
216      (milk)                              (eggs)              0.1296   
264      (milk)                      (french fries)              0.1296   
320      (milk)                 (frozen vegetables)              0.1296   
383      (milk)                       (ground beef)              0.1296   
42       (milk)                           (burgers)              0.1296   
453      (milk)                            (shrimp)              0.1296   
358      (milk)                         (green tea)              0.1296   
446      (milk)                         (olive oil)              0.1296   
448      (milk)                          (pancakes)      